[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dhupee/Bangkit-C22CB-Company-Based-Capstone/blob/30b0995970f29114749cff04deef444de6832993/ML/distilbert_transfer_learn.ipynb)

In [22]:
# check python version
import sys
print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [23]:
# notebook settings

COLAB_MODE = False # set to True if running in Google Colab
ENABLE_JSON2CSV = False # set to True if you want to convert json dataset to csv

In [24]:
# if COLAB_MODE is True, then work around the repository
if COLAB_MODE:
    import os
    branch_name = 'dhupee-dev'
    cloned_repo_name = 'remote-clone'
    target_repo_dir = '/content/remote-clone/ML'
    repo_link = 'https://github.com/dhupee/Bangkit-C22CB-Company-Based-Capstone.git'
    # if current directory is not the cloned repo, clone it
    if not os.path.exists(target_repo_dir):
        !git clone --single-branch --branch $branch_name $repo_link $cloned_repo_name
        print('Repo successfully cloned!')
        %cd $target_repo_dir
        %pwd
    else:
        print('Repo already cloned')

In [25]:
# check if transformers and tensorflow are installed, if not install them
# use transformers version 4.18.0 and tensorflow version 2.8.0
try:
    import transformers
    import tensorflow as tf
    print("transformers and tensorflow are installed")
except:
    print("transformers and tensorflow are not installed")
    print("installing transformers and tensorflow")
    # install transformers 4.18.0 and tensorflow 2.8.0
    %pip install transformers==4.18.0
    %pip install tensorflow==2.8.1
    # import transformers and tensorflow again
    import transformers
    import tensorflow as tf

transformers and tensorflow are installed


In [26]:
model_name = "cahya/bert-base-indonesian-522M"
batch_size = 32

from transformers import BertTokenizer, TFAutoModel # make sure use tensorflow model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

: 

: 

In [ ]:
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
Total params: 110,617,344
Trainable params: 110,617,344
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# test tokenizer
tokenizer("Nama kamu siapa?")

{'input_ids': [3, 1769, 8343, 6186, 32, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer("saya suka makan nasi goreng")

{'input_ids': [3, 3245, 5366, 2464, 6014, 11186, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
unmasker = transformers.pipeline('fill-mask', model = model_name)
unmasker("mainan saya [MASK] di jalan")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


[{'score': 0.0840364545583725,
  'token': 2186,
  'token_str': 'berada',
  'sequence': 'mainan saya berada di jalan'},
 {'score': 0.07038316130638123,
  'token': 1821,
  'token_str': 'ada',
  'sequence': 'mainan saya ada di jalan'},
 {'score': 0.0403575636446476,
  'token': 1998,
  'token_str': 'sendiri',
  'sequence': 'mainan saya sendiri di jalan'},
 {'score': 0.029048316180706024,
  'token': 2444,
  'token_str': 'lahir',
  'sequence': 'mainan saya lahir di jalan'},
 {'score': 0.028137197718024254,
  'token': 3812,
  'token_str': 'berdiri',
  'sequence': 'mainan saya berdiri di jalan'}]

In [49]:
# load dataset json file
import json

train_json_dir = "Translated/train-v2.0_indo.json"
dev_json_dir = "Translated/dev-v2.0_indo.json"
tester_json_dir  = "Translated/tester_indo.json"

dataset_dirs = [train_json_dir, dev_json_dir, tester_json_dir]
# dataset_dirs = [tester_json_dir]

In [ ]:
'''
    This function is for converting SQuAD json file to pandas dataframe, iteratively

    I dont want run this locally, better use colab
'''

if ENABLE_JSON2CSV:
    import utils
    for dir in dataset_dirs:
        with open(dir, encoding="utf-8") as json_file:
            file = json.load(json_file)
            dict_file = file
            data = dict_file['data']

        df = utils.json_to_df(data)
        df.to_csv(dir.replace(".json", ".csv"), index = False)

---

In [ ]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

In [ ]:
try:
    import pandas as pd
except:
    print("pandas is not installed")
    print("installing pandas...")
    %pip install pandas
    import pandas as pd

train_csv_dir = "Translated/train-v2.0_indo.csv"
dev_csv_dir = "Translated/dev-v2.0_indo.csv"
tester_csv_dir  = "Translated/tester_indo.csv"

# open one of the csv file
df_tester = pd.read_csv(tester_csv_dir)
df_tester.head()

,input_id,title,context,question,answer_text,answer_start
0,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
1,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
2,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
3,56ddde6b9a695914005b9628,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Di negara manakah Normandia berada?,Perancis,159.0
4,56ddde6b9a695914005b9629,orang Normandia,"Gaya pound memiliki padanan metrik, lebih jara...",Kapan orang Normandia di Normandia?,abad 10 dan 11,94.0


In [ ]:
#check dataframe datatype
df_tester.dtypes

input_id         object
title            object
context          object
question         object
answer_text      object
answer_start    float64
dtype: object

In [ ]:
csv_dirs = [train_csv_dir, dev_csv_dir, tester_csv_dir]

# check every csv file, if there is feature longer than max_length, then print it
for csv_dir in csv_dirs:
    df = pd.read_csv(csv_dir)
    longer_feature = 0
    for index, row in df.iterrows():
        if len(row[['context', 'question', 'input_id']]) > max_length:
            longer_feature += 1
            print(f"{csv_dir} has {longer_feature} features longer than {max_length}")
            print("\n")
    if longer_feature == 0:
        print(f"{csv_dir} has no features longer than {max_length}")
        print("\n")

Translated/train-v2.0_indo.csv has no features longer than 384


Translated/dev-v2.0_indo.csv has no features longer than 384


Translated/tester_indo.csv has no features longer than 384




In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [17]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

# enable fine-tuning
model.trainable = True

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
learning_rate = 0.00001
num_train_epochs = 2
weight_decay = 0.01

In [19]:
# TODO: load preprocessed dataset
# TODO: train model
# TODO: evaluate model
# TODO: save model

In [2]:
try:
    from datasets import load_dataset, load_metric
except:
    print("datasets is not installed")
    print("installing datasets...")
    %pip install datasets
    from datasets import load_dataset, load_metric

datasets = load_dataset("squad_v2")

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 333.41it/s]


Dataset squad_v2 downloaded and prepared to C:\Users\Arya Wijna\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 24.08it/s]


In [53]:
# sample of dataset randomly
import random
dataset_sample = datasets["train"][random.randint(0, len(datasets["train"]))]
dataset_sample

{'id': '573443f3acc1501500babd40',
 'title': 'Richmond,_Virginia',
 'context': 'Richmond is also fast-becoming known for its food scene, with several restaurants in the Fan, Church Hill, Jackson Ward and elsewhere around the city generating regional and national attention for their fare. Departures magazine named Richmond "The Next Great American Food City" in August 2014. Also in 2014, Southern Living magazine named three Richmond restaurants – Comfort, Heritage and The Roosevelt – among its "100 Best Restaurants in the South", while Metzger Bar & Butchery made its "Best New Restaurants: 12 To Watch" list. Craft beer and liquor production is also growing in the River City, with twelve micro-breweries in city proper; the oldest is Legend Brewery, founded in 1994. Three distilleries, Reservoir Distillery, Belle Isle Craft Spirits and James River Distillery, were established in 2010, 2013 and 2014, respectively.',
 'question': 'What was the first microbrewery to set up shop in Richmond?'

In [ ]:
# take sample title
title = dataset_sample["title"]
print(title)

In [36]:
# translating squad_v2 from english to indonesia

try:
    from deep_translator import GoogleTranslator
except:
    print("deep_translator is not installed")
    print("installing deep_translator...")
    %pip install deep_translator
    from deep_translator import GoogleTranslator

In [55]:
lang_source = 'en'
lang_dest = 'id'

def translate_squad_v2(dataset, lang_source, lang_dest):
    '''translate squad_v2 from english to indonesia'''
    translator = GoogleTranslator(lang_source, lang_dest)
    for i in range (len(datasets['train'])):
        dataset['train'][i]['title'] = translator.translate(datasets['train'][i]['title'])
        dataset['train'][i]['context'] = translator.translate(datasets['train'][i]['context'])
        dataset['train'][i]['question'] = translator.translate(datasets['train'][i]['question'])
    return dataset

In [56]:
translated_dataset = translate_squad_v2(datasets, lang_source, lang_dest)

KeyboardInterrupt: 